In [1]:
%load_ext autoreload
%autoreload 2

In [6]:
import os
import pandas as pd

from transformers import AutoModelForCausalLM
from transformers import AutoTokenizer

from battleship.translator import Translator

In [3]:
HF_AUTH_TOKEN = os.environ.get("HF_AUTH_TOKEN")
MODEL_NAME = "WizardLM/WizardCoder-Python-7B-V1.0"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, token=HF_AUTH_TOKEN)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME, token=HF_AUTH_TOKEN, device_map="auto"
)

Loading checkpoint shards: 100%|██████████| 2/2 [01:46<00:00, 53.42s/it]


In [9]:
translator = Translator(model=model, tokenizer=tokenizer)

In [10]:
completion = translator.question_to_code("How many blocks is the blue ship?")
print(completion)

(size Blue)


In [11]:
completion = translator.question_to_code("What is at A7?")
print(completion)

(color A-7)
